# Structurer et explorer des données textuelles

Notebook Introduction au traitement du langage naturel - 15/05/2025 - Émilien Schultz

## Les données

Données questions au gouvernement https://www.data.gouv.fr/fr/datasets/questions-au-gouvernement/

Télécharger les données et les décompresser dans un répertoire `data/` dans l'espace de travail

## Les bibliothèques

- `pandas` pour la manipulation de données
- `nltk` pour le traitement de texte
- `matplotlib` pour la visualisation
- `scikit-learn` pour le traitement de texte et la modélisation


In [ ]:
#!pip install pandas nltk scikit-learn matplotlib

## Structurer les données

Avoir des données facilement manipulables

In [ ]:
import pandas as pd
import glob
import json

In [ ]:
fichiers = glob.glob("../data/json/*")
fichiers[0]

In [ ]:
fichier = json.load(open(fichiers[100], "r"))
fichier["question"]

On veut récupérer un tableau avec :

- la date de la question
- la législature
- le texte de la question

In [ ]:
numero = fichier["question"]["identifiant"]["numero"],
legislature = fichier["question"]["identifiant"]["legislature"],
date = fichier["question"]["textesReponse"]["texteReponse"]["infoJO"]["dateJO"]
texte = fichier["question"]["textesReponse"]["texteReponse"]["texte"]

In [ ]:
def get_info(fichier):
    """
    Fonction qui extrait les informations d'une question parlementaire
    """
    try:
        numero = fichier["question"]["identifiant"]["numero"]
        legislature = fichier["question"]["identifiant"]["legislature"]
        date = fichier["question"]["textesReponse"]["texteReponse"]["infoJO"]["dateJO"]
        texte = fichier["question"]["textesReponse"]["texteReponse"]["texte"]
        return {
            "numero": numero,
            "legislature": legislature,
            "date": date,
            "texte": texte
        }
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations : {e}")
        return None

Application au corpus

In [ ]:
t = [get_info(json.load(open(fichier, "r"))) for fichier in fichiers]
t = [i for i in t if i is not None]
df = pd.DataFrame(t)

## Nettoyer les données

Enlever les balises HTML et les autres soucis. 

Pour cela on peut utiliser une regex. 

### Remarque sur les regex

- bibliothèque `re` ou `regex`
- `re.sub` pour remplacer une partie d'une chaîne par une autre
- `re.findall` pour trouver toutes les occurrences d'une regex dans une chaîne
- [Aller voir une cheatsheet 🤓](https://www.pythoncheatsheet.org/cheatsheet/regular-expressions) ou utiliser un [site dédié](https://regex101.com/)

In [ ]:
import re
re.sub(r"<.*?>", "", "Ceci est un <b>test</b>")

Appliquer au corpus

In [ ]:
def clean_text(text):
    """
    Fonction qui nettoie le texte en supprimant les balises HTML et les espaces inutiles
    :param text: le texte à nettoyer
    :return: le texte nettoyé
    """
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s\s+", " ", text)
    return text.strip()

Application au corpus

In [ ]:
df["texte"] = df["texte"].apply(clean_text)
df["texte"].head(5)

In [ ]:
df.to_csv("../data/dataframe.csv")

## Analyse à l'échelle des mots

### Chercher la présence d'un mot

Les bases de la fouille de données. Quels sont les questions qui parlent d'intelligence artificielle ?

In [ ]:
df["texte"].str.lower().str.contains("intelligence artificielle").sum()

Et de science ?

In [ ]:
df["texte"].str.lower().str.contains("science|scientifique").sum()

Faire une recherche sur toutes les variables possibles de l'IA

### Tokenisation

Découper un texte

#### Utiliser les regex

In [ ]:
import re
word_pattern = r"\w+"
tokens = re.findall(word_pattern, "Ceci est un test")
tokens

#### Utiliser une première bibliothèque : `nltk`

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

word_tokenize("Ceci est un test")

### Quels sont les termes les plus fréquents ?

### Quelles sont les expressions qui reviennent le plus souvent ?

Utilisons les bigrammes et les trigrammes

In [ ]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def generate_bigrams_nltk(text):
    tokens = word_tokenize(text.lower())
    bigrams = list(ngrams(tokens, 2))
    return bigrams

generate_bigrams_nltk(df["texte"].iloc[0])

#### Enlever les stop words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

nltk.download("stopwords")
french_stopwords = list(set(stopwords.words("french")))
french_stopwords[0:10]


def generate_bigrams_nltk(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in french_stopwords]
    bigrams = list(ngrams(filtered_tokens, 2))
    return bigrams

generate_bigrams_nltk(df["texte"].iloc[0])


In [ ]:
# Count bigrams:
vectorizer = CountVectorizer(stop_words=french_stopwords, ngram_range=(3, 3), max_features=300)
trigrams = (
    pd.DataFrame(
        vectorizer.fit_transform(df["texte"]).toarray(),
        columns=vectorizer.get_feature_names_out(),
    )
    .T.sum(axis=1)
    .sort_values(ascending=False)
)


## Représenter les textes

### Vecteur brut

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=french_stopwords, ngram_range=(1, 1), max_features=800)
X = vectorizer.fit_transform(df["texte"])
X

### Une version un peu plus avancée

- Term Frequency-Inverse Document Frequency
    - Amélioration du DTM
- Approche souvent utilisée pour mettre en valeur les mots les plus spécifiques
- `Scikit-learn` a `TfidfVectorizer`

$$\text{TF-IDF}(t, d, D) = \left( \frac{f_{t,d}}{n_d} \right) \times \log \left(\frac{N}{\text{df}_t} \right)
$$

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=french_stopwords, ngram_range=(1, 1), max_features=800)
X = vectorizer.fit_transform(df["texte"])
X.shape

In [ ]:
len(vectorizer.get_feature_names_out())

Faire la matrice TF-IDF, identifier les mots qui ont le score le plus important

## Distance entre deux textes

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

cosine_similarity(X[0], X[2])

In [ ]:
distances = pd.DataFrame(pairwise_distances(X, metric="cosine"))

In [ ]:
distances[10].idxmax()

## Application : Faire un nuage de mots avec WordCloud

Un coup d'oeil à la [documentation](https://amueller.github.io/word_cloud/)